In [1]:
import cv2
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
# df = pd.read_csv("../data_generated/list_boxing_person01_s1.csv")
# df.shape
# df.iloc[0,0:384]

In [3]:
# TRAIN_PEOPLE_ID = [11, 12, 13, 14, 15, 16, 17, 18]
# DEV_PEOPLE_ID = [19, 20, 21, 23, 24, 25, 1, 4]
# TEST_PEOPLE_ID = [22, 2, 3, 5, 6, 7, 8, 9, 10]
TRAIN_PEOPLE_ID = [1, 2, 3, 4, 7, 6, 8,11, 12, 13, 14, 15, 16, 17, 18]
TEST_PEOPLE_ID = [5]

In [4]:
# files = os.listdir("../data_generated")

In [10]:
data_gen_path = "../data_generated"
#os.listdir(data_gen_path)
no_cuboids = 48
feature_lenght = 192
feature_lenght_all = 2*feature_lenght

def concatenate_features(path, flac_all=True):
    files = os.listdir(path)
    features_for_clusters = []
    
    angles = []
    magnitudes = []
    feature_=[]
    
    train = []
    test = []
    
    print('Nº files:', len(files))
    
    
    for idx,file in enumerate(files):
        person_id = int(file.split("_")[2][6:])
#         print(person_id)
        video_features = []
        pt = os.path.join(data_gen_path,file)
#         print('processing video_csv: ',idx," file:",pt)
    
        
        df = pd.read_csv(pt)
        for i in range(no_cuboids): #for each row
            
            f_angles = (df.iloc[i,0:feature_lenght]) #first 192 columns
            f_magnitudes = (df.iloc[i,feature_lenght:2*feature_lenght])  #last 192 columns
#             print("type angles: ",type(f_angles))
            angles.append(np.array(f_angles,dtype='float'))
            magnitudes.append(np.array(f_magnitudes,dtype='float'))
            #add features for kmeans
            features_for_clusters.append(np.array(f_magnitudes,dtype='float'))
            
        video_features.append(magnitudes)
        
        if person_id in TRAIN_PEOPLE_ID:
            train.append({
                "video": file,
                "category": df.activity[0], 
                "features": video_features
            })
        else: test.append({
                "video": file,
                "category": df.activity[0], 
                "features": video_features
            })
          
    print('feature len:', len(video_features))
    print('train len:', len(train))
    print('test len:', len(test))
    print("features_for_clusters: ",len(features_for_clusters))
    return train, test, angles, magnitudes, features_for_clusters

train, test ,angles,magnitudes, features_for_clusters = concatenate_features(data_gen_path)
# pickle.dump(train, open("../pickle/train.p", "wb"))
# pickle.dump(test, open("../pickle/test.p", "wb"))


Nº files: 597
feature len: 1
train len: 358
test len: 239
features_for_clusters:  28656


In [12]:
# train[0]["features"]
features_for_clusters
# angles

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [13]:
n_clusters = 100

from sklearn.cluster import KMeans
from numpy import size

# Clustering with KMeans.
print("Running KMeans clustering")
kmeans = KMeans(init='k-means++', n_clusters=n_clusters, n_init=10, n_jobs=2,
    verbose=1)
kmeans.fit(features_for_clusters)
# Save trained kmeans object to file.
pickle.dump(kmeans, open("../pickle/cb_%dclusters.p" % n_clusters, "wb"))

Running KMeans clustering
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 1.5163002126684957e+24
start iteration
done sorting
end inner loop
Iteration 1, inertia 1.4430529834841979e+24
start iteration
done sorting
end inner loop
Iteration 2, inertia 1.4211314033943202e+24
start iteration
done sorting
end inner loop
Iteration 3, inertia 1.4161420113192564e+24
start iteration
done sorting
end inner loop
Iteration 4, inertia 1.4127478202102732e+24
start iteration
done sorting
end inner loop
Iteration 5, inertia 1.4097686800847538e+24
start iteration
done sorting
end inner loop
Iteration 6, inertia 1.4085618648134035e+24
start iteration
done sorting
end inner loop
Iteration 7, inertia 1.4083661671933034e+24
start iteration
done sorting
end inner loop
Iteration 8, inertia 1.4083395334022216e+24
start iteration
done sorting
end inner loop
Iteration 9, inertia 1.4083395334022216e+24
center shift 0.000000e+00 within tolerance 5.607699e+14
Initialization

Iteration 7, inertia 1.411072442129837e+24
start iteration
done sorting
end inner loop
Iteration 8, inertia 1.4110155480608959e+24
start iteration
done sorting
end inner loop
Iteration 9, inertia 1.4109977199600307e+24
start iteration
done sorting
end inner loop
Iteration 10, inertia 1.4109977199600307e+24
center shift 0.000000e+00 within tolerance 5.607699e+14
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 1.469162939990611e+24
start iteration
done sorting
end inner loop
Iteration 1, inertia 1.425860125489958e+24
start iteration
done sorting
end inner loop
Iteration 2, inertia 1.4138257125954644e+24
start iteration
done sorting
end inner loop
Iteration 3, inertia 1.407679027423996e+24
start iteration
done sorting
end inner loop
Iteration 4, inertia 1.4055105992603548e+24
start iteration
done sorting
end inner loop
Iteration 5, inertia 1.405139530699149e+24
start iteration
done sorting
end inner loop
Iteration 6, inertia 1.405016008021185e+24
s

## Building Bag of Words

In [14]:
from scipy.cluster.vq import vq

def make_bow(dataset, clusters):
    n_videos = len(dataset)
    bow = np.zeros((n_videos, clusters.shape[0]), dtype=np.float)
    # Make bow vectors for all videos.
    video_index = 0
    for video in dataset:
        visual_word_ids = vq(video["features"], clusters)[0]
        #print(len(visual_word_ids))
        for word_id in visual_word_ids:
            bow[video_index, word_id] += 1
        video_index += 1
        
    # Check whether to use TF-IDF weighting.
    tfidf = False
    if tfidf:
        print("Applying TF-IDF weighting")
        freq = np.sum((bow > 0) * 1, axis = 0)
        idf = np.log((n_videos + 1) / (freq + 1))
        bow = bow * idf

    # Replace features in dataset with the bow vector we've computed.
    video_index = 0
    for i in range(len(dataset)): #for each video
        dataset[i]["features"] = bow[video_index]
        video_index += 1

        #if (i + 1) % 50 == 0:
        #    print("Processed %d/%d videos" % (i + 1, len(dataset)))
    print("reeplaced features by bow...")
    return dataset


dataset_train  = "../pickle/train.p"
dataset_test  = "../pickle/test.p"
codebook_file = "../pickle/cb_%dclusters.p"%n_clusters
# Load clusters.
codebook = pickle.load(open(codebook_file, "rb"))
clusters = codebook.cluster_centers_

# Load dataset.
dataset_train = pickle.load(open(dataset_train, "rb"))
dataset_test = pickle.load(open(dataset_test, "rb"))

train_bow = make_bow(dataset_train, clusters)
test_bow = make_bow(dataset_test, clusters)
#len(test_bow), train_bow

reeplaced features by bow...
reeplaced features by bow...


In [15]:
clusters[0].shape, len(dataset_train[0]["features"])

((192,), 100)

In [16]:
dataset_train[0]["features"].shape

(100,)

## Train

In [17]:
from sklearn.svm import SVC

def make_dataset(dataset):
    X = []
    Y = []

    for video in dataset:
        X.append(video["features"])
        Y.append(video["category"])
    return X,Y
        
X_train, Y_train = make_dataset(dataset_train)
X_test, Y_test = make_dataset(dataset_test)

In [ ]:
len(X_train[0]),len(X_test)

In [ ]:
X_train

In [18]:
# Train SVM and save to file.
clf = SVC(C=1, kernel="linear", verbose=True)
clf.fit(X_test, Y_test)

[LibSVM]

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=True)

## train

In [21]:
from sklearn.svm import SVC

def make_dataset(dataset):
    X = []
    Y = []

    for video in dataset:
        X.append(video["features"])
        Y.append(video["category"])
    return X,Y
        
X, Y = make_dataset(dataset_train)
# Train SVM and save to file.
clf = SVC(C=1, kernel="linear",verbose=True)
clf.fit(X, Y)


[LibSVM]

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=True)

## Prediction

In [22]:
CATEGORIES = ["boxing", "handclapping", "handwaving", "jogging", "running", 
    "walking"]

def evaluate(dataset):
    correct = 0
    confusion_matrix = np.zeros((6, 6))
    
    
    for video in dataset:    
        #video = dataset[0]
        predicted = clf.predict([video["features"]])
    #     print(predicted)
        # Check if majority is correct.
        if predicted == video["category"]:
            correct += 1
#     print("%d/%d Correct" % (correct, len(dataset)))
    print("Accuracy =", correct / len(dataset))

evaluate(dataset_test)

Accuracy = 0.20920502092050208


In [ ]:

from sklearn import metrics
y_est = clf.predict(X_test)
print(metrics.accuracy_score(Y_test,y_est))


In [ ]:
X_test = dataset[1]['features']
Y_test = dataset[1]['category']

In [ ]:
X_test = X
Y_test = Y

In [ ]:
Y_pred = clf.predict(X)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_pred, Y_test)